In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
# Defining catalog and schema name
catalog_name = "walmart_purchases"
silver_schema = f"{catalog_name}.silver"
gold_schema = f"{catalog_name}.gold"

silver_table = f"{silver_schema}.silver_purchases"

## Dim Customers

In [0]:
%sql
CREATE OR REPLACE TABLE walmart_purchases.gold.dim_customers
AS
WITH dim_cust AS 
(
  SELECT
    DISTINCT(Customer_ID),
    Age,
    Gender,
    City,
    Repeat_Customer
  FROM walmart_purchases.silver.silver_purchases
)
SELECT *, row_number() OVER (ORDER BY Customer_ID) as DimCustomerKey,
      current_date() as start_date,
      CAST('9999-12-31' AS DATE) as end_date,
      'Y' as is_current
FROM dim_cust

In [0]:
# Adding additional columns for SCD Type 2
spark.sql("""SELECT *,
        current_timestamp as start_date,
        CAST('3000-01-01' AS TIMESTAMP) as end_date,
        'Y' as is_current
FROM walmart_purchases.silver.silver_purchases""").createOrReplaceTempView("src")

## SCD Type -2

In [0]:
%sql
MERGE INTO walmart_purchases.gold.dim_customers tgt
USING src
ON src.Customer_ID = tgt.Customer_ID AND tgt.is_current = 'Y'
WHEN MATCHED AND (
    tgt.City <> src.City
) THEN
  UPDATE SET
    tgt.is_current = 'N',
    tgt.End_Date = current_date()
WHEN NOT MATCHED THEN
  INSERT (
    Customer_ID,
    Age,
    Gender,
    City,
    Repeat_Customer,
    start_date,
    end_date,
    is_current
  )
  VALUES (
    src.Customer_ID,
    src.Age,
    src.Gender,
    src.City,
    src.Repeat_Customer,
    current_date(),
    '9999-12-31',
    'Y'
  )
    

## Dim Products

In [0]:
%sql
CREATE OR REPLACE TABLE walmart_purchases.gold.dim_products
AS
WITH dim_prod AS 
(
  SELECT
    DISTINCT(Category),
    Product_Name
  FROM walmart_purchases.silver.silver_purchases
)
SELECT *, row_number() OVER (ORDER BY Category, Product_Name) as DimProductKey,
        current_date() as start_date,
        CAST('9999-12-31' AS DATE) as end_date,
        'Y' as is_current
FROM dim_prod

In [0]:
%sql
MERGE INTO walmart_purchases.gold.dim_products tgt
USING src
ON src.Product_Name = tgt.Product_Name AND tgt.is_current = 'Y'
WHEN MATCHED AND tgt.Category <> src.Category THEN
UPDATE SET 
    tgt.is_current = 'N',
    tgt.End_Date = current_date()
WHEN NOT MATCHED THEN
  INSERT (
    Category,
    Product_Name,
    start_date,
    end_date,
    is_current
  )
  VALUES (
    src.Category,
    src.Product_Name,
    current_date(),
    '9999-12-31',
    'Y'
  )

In [0]:
%sql
SELECT * FROM walmart_purchases.gold.dim_products

## Dim Date

In [0]:
%sql
CREATE OR REPLACE TABLE walmart_purchases.gold.dim_date
WITH dim_date AS
(
  SELECT 
    DISTINCT(Purchase_Date),
    Year,
    Month,
    Day,
    Week 
  FROM walmart_purchases.silver.silver_purchases
)
SELECT *, row_number() OVER (ORDER BY Purchase_Date) as DimDateKey
FROM dim_date;

### MERGE for Incremental Load

In [0]:
%sql
MERGE INTO walmart_purchases.gold.dim_date tgt
USING walmart_purchases.silver.silver_purchases src
ON tgt.Purchase_Date = src.Purchase_Date
WHEN NOT MATCHED THEN
  INSERT(
    Purchase_Date,
    Year,
    Month,
    Day,
    Week
  )
  VALUES (
    src.Purchase_Date,
    src.Year,
    src.Month,
    src.Day,
    src.Week
  )

## Dim Payment

In [0]:
%sql
CREATE OR REPLACE TABLE walmart_purchases.gold.dim_payment
WITH dim_payment AS
(
  SELECT DISTINCT(Payment_Method)
  FROM walmart_purchases.silver.silver_purchases
)
SELECT *, row_number() OVER (ORDER BY Payment_Method) as DimPaymentKey
FROM dim_payment;

In [0]:
%sql
MERGE INTO walmart_purchases.gold.dim_payment tgt
USING walmart_purchases.silver.silver_purchases src
ON src.Payment_Method = tgt.Payment_Method
WHEN NOT MATCHED THEN
  INSERT(
    Payment_Method
  )
  VALUES (
    src.Payment_Method
  )

## Fact Purchases

In [0]:
%sql
CREATE OR REPLACE TABLE walmart_purchases.gold.fact_purchases
AS
SELECT 
  C.DimCustomerKey,
  P.DimProductKey,
  D.DimDateKey,
  PY.DimPaymentKey,
  F.Purchase_Amount,
  F.Discount_Applied,
  F.Rating
FROM walmart_purchases.silver.silver_purchases F
LEFT JOIN walmart_purchases.gold.dim_customers C
ON F.Customer_ID = C.Customer_ID
LEFT JOIN walmart_purchases.gold.dim_products P
ON F.Category = P.Category
LEFT JOIN walmart_purchases.gold.dim_date D
ON D.Purchase_Date = F.Purchase_Date
LEFT JOIN walmart_purchases.gold.dim_payment PY
ON F.Payment_Method = PY.Payment_Method

In [0]:
%sql
SELECT * FROM walmart_purchases.gold.fact_purchases